## OpenMMTools Integration

This enables OPS to use any testsystem from openmmtools to be used as well as add ANY possible OpenMM setup, not just the ones that we provided before. The old OpenMMEngine has been rename intp `SimpleOpenMMEngine`

In [1]:
import openpathsampling as paths
import openmmtools as omt
import simtk.openmm as omm
import simtk.unit as u
import mdtraj as md

import openpathsampling.engines.openmm as eng

Create Alanine in Vacuum and run it using OPS.

In [2]:
testsystem = omt.testsystems.AlanineDipeptideVacuum()

Let's have a look at the content

In [3]:
#! skip
{ key: type(value) for key, value in testsystem.__dict__.items()}

{'_positions': simtk.unit.quantity.Quantity,
 '_system': simtk.openmm.openmm.System,
 '_topology': simtk.openmm.app.topology.Topology}

An `OpenMM` simulation in OPS needs 3 ingredients to function
1. A template snapshot (which contains topolgy and coordinates/velocities), 
2. an OpenMM system object, and 
3. an OpenMM integrator object.

In the following we will get these parts.

#### 1. The template

The information for the template is contained in the `_topology` and the `_positions` object.

In [4]:
template = eng.snapshot_from_testsystem(testsystem)

#### 2. The system

This is easy since we directly get it from the testsytem objects

In [5]:
system = testsystem.system

#### 3. The integrator

The openmmtools system is (almost) independent of the integrator and `openmm` provides us with lots of options. For now we pick something simple.

In [6]:
integrator = omm.VerletIntegrator(
    0.002 * u.picoseconds    
)

build the engine

In [7]:
engine = eng.Engine(template, system, integrator, options={'platform': 'CPU'})

Let's run a simulation of 10 steps.

In [8]:
traj = engine.generate(engine.template, [paths.LengthEnsemble(10).can_append])

In [9]:
print traj

Trajectory[10]


In [10]:
#! skip
print traj[5].coordinates

[[ 0.18766736  0.11599679  0.01245004]
 [ 0.19458341  0.22444927  0.00401414]
 [ 0.1443672   0.27122689  0.08869714]
 [ 0.14886747  0.25747473 -0.08926164]
 [ 0.33994537  0.26638274  0.00609843]
 [ 0.42709578  0.1897011   0.04380051]
 [ 0.36886493  0.38998331 -0.03181561]
 [ 0.293367    0.452431   -0.05633815]
 [ 0.49859312  0.4565761  -0.02400124]
 [ 0.5581438   0.41515287  0.0573552 ]
 [ 0.5732535   0.43391328 -0.15561179]
 [ 0.5191127   0.48312774 -0.23640657]
 [ 0.67427683  0.47320502 -0.14414124]
 [ 0.58423055  0.32728151 -0.17536463]
 [ 0.4749522   0.60669039 -0.00127668]
 [ 0.359584    0.64924202  0.00474684]
 [ 0.5835927   0.68379167  0.00902624]
 [ 0.6747424   0.64066121  0.00332412]
 [ 0.57127444  0.82897221  0.01853087]
 [ 0.46718868  0.86095438  0.02347778]
 [ 0.62214258  0.86301533  0.10872177]
 [ 0.61715852  0.87434251 -0.06931573]] nm


In [11]:
psi = md.compute_psi(traj.md())

In [12]:
#! ignore
psi[1][3:8]

array([[ 3.10614538],
       [-3.09586287],
       [ 3.13675523],
       [ 3.12832713],
       [ 3.09317875]], dtype=float32)

In [13]:
st = paths.Storage('engine_store_test.nc', mode='w', template=template)

In [14]:
st.engines.save(engine);

The engine is stored by using the XML serialization of OpenMM and put this into the storage.

In [15]:
#! skip
st.variables['engines_json'][0][0:256] + '...'

u'{"_cls": "OpenMMEngine", "_dict": {"integrator_xml": "<?xml version=\\"1.0\\" ?>\\n<Integrator constraintTolerance=\\"1e-05\\" stepSize=\\".002\\" type=\\"VerletIntegrator\\" version=\\"1\\"/>\\n", "options": {"timestep": null, "n_frames_max": 5000, "platform": "CPU",...'

In [16]:
st.close()

### try to load the stored engine

In [17]:
st = paths.AnalysisStorage('engine_store_test.nc')

In [18]:
engine = st.engines[0]

Run another 10 steps to test

In [19]:
traj = engine.generate(engine.template, [paths.LengthEnsemble(10).can_append])

In [20]:
print traj

Trajectory[10]


In [21]:
st.close()